In [1]:
# Import Libraries
import numpy as np
import cv2
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from tqdm import tqdm
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping
import os
from sklearn.preprocessing import LabelEncoder
import pickle

2024-05-18 15:44:41.365109: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 15:44:41.365194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 15:44:41.367042: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load Data
labels = ['glioma_tumor', 'no_tumor', 'meningioma_tumor', 'pituitary_tumor']

X_train = []
y_train = []
image_size = 150
for i in labels:
    folderPath = os.path.join('../input/brain-tumor-classification-mri','Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)

X_test = []
y_test = []

for i in labels:
    folderPath = os.path.join('../input/brain-tumor-classification-mri','Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_test.append(img)
        y_test.append(i)

100%|██████████| 74/74 [00:00<00:00, 142.33it/s]


In [3]:
# Shuffle data
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42) 

In [4]:
# Convert strings to numeric
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

In [5]:
# Convert to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# print shapes
print("Train Images Shape:", X_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Images Shape: ", X_test.shape)
print("Test Labels Shape: ", y_test.shape)

Train Images Shape: (2870, 150, 150, 3)
Train Labels Shape: (2870,)
Test Images Shape:  (394, 150, 150, 3)
Test Labels Shape:  (394,)


In [6]:
# Display class distribution
unique_values, counts = np.unique(y_train, return_counts=True)
value_counts = dict(zip(unique_values, counts))
print(value_counts)

{0: 826, 1: 822, 2: 395, 3: 827}


In [7]:
# Calculate class weights and store in dictionary
class_weights = compute_class_weight('balanced', classes=unique_values, y=y_train)
class_weights_dict = dict(zip(labels, class_weights))

# Print class weights
print("Class Weights Dictionary:")
for key, value in class_weights_dict.items():
    print(f"Class {key}: {value}")

Class Weights Dictionary:
Class glioma_tumor: 0.8686440677966102
Class no_tumor: 0.8728710462287105
Class meningioma_tumor: 1.8164556962025316
Class pituitary_tumor: 0.8675937122128174


In [8]:
# One hot encode
y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(labels))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(labels))

In [9]:
# Load EfficientNetB0 Model
effnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))
model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)
model.summary()
model.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 150, 150,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 150, 150,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 150, 150,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 151, 151,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 75, 75,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 75, 75,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 75, 75,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 75, 75,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 75, 75,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 75, 75,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 75, 75,    │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 75, 75,    │        512 │ block1a_se_excit

 Total params: 4,054,695 (15.47 MB)

 Trainable params: 4,012,672 (15.31 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [10]:
# Train model
history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=20,
                    verbose=2,
                    batch_size=32,
                    class_weight=class_weights_dict)

Epoch 1/20


I0000 00:00:1716047196.967227    2439 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


81/81 - 175s - 2s/step - accuracy: 0.8358 - loss: 0.4341 - val_accuracy: 0.8362 - val_loss: 0.5139
Epoch 2/20
81/81 - 6s - 71ms/step - accuracy: 0.9361 - loss: 0.1777 - val_accuracy: 0.8467 - val_loss: 0.6507
Epoch 3/20
81/81 - 6s - 71ms/step - accuracy: 0.9655 - loss: 0.1034 - val_accuracy: 0.7944 - val_loss: 0.8031
Epoch 4/20
81/81 - 6s - 71ms/step - accuracy: 0.9721 - loss: 0.0843 - val_accuracy: 0.9164 - val_loss: 0.3197
Epoch 5/20
81/81 - 6s - 71ms/step - accuracy: 0.9853 - loss: 0.0387 - val_accuracy: 0.8850 - val_loss: 0.4680
Epoch 6/20
81/81 - 6s - 70ms/step - accuracy: 0.9779 - loss: 0.0677 - val_accuracy: 0.9059 - val_loss: 0.4355
Epoch 7/20
81/81 - 6s - 71ms/step - accuracy: 0.9826 - loss: 0.0476 - val_accuracy: 0.9582 - val_loss: 0.1368
Epoch 8/20
81/81 - 6s - 71ms/step - accuracy: 0.9888 - loss: 0.0422 - val_accuracy: 0.6516 - val_loss: 2.0808
Epoch 9/20
81/81 - 6s - 71ms/step - accuracy: 0.9888 - loss: 0.0335 - val_accuracy: 0.8780 - val_loss: 0.5970
Epoch 10/20
81/81 - 6

In [11]:
# Map predicted and true label indices to label names
predicted_labels = [labels[index] for index in np.argmax(model.predict(X_test), axis=1)]
true_labels = [labels[index] for index in np.argmax(y_test, axis=1)]

# Generate classification report
print(classification_report(true_labels, predicted_labels))

13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 705ms/step
                  precision    recall  f1-score   support

    glioma_tumor       1.00      0.25      0.40       100
meningioma_tumor       0.82      0.98      0.90       105
        no_tumor       0.61      0.97      0.75       115
 pituitary_tumor       1.00      0.82      0.90        74

        accuracy                           0.76       394
       macro avg       0.86      0.76      0.74       394
    weighted avg       0.84      0.76      0.73       394

